In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install -q datasets accelerate scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.4 MB/s eta 0:00:00


In [3]:
!pip uninstall -y transformers
!pip install transformers --upgrade --no-cache-dir


Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 149.4 MB/s eta 0:00:00


In [4]:
import transformers
print(transformers.__version__)


4.52.4


In [5]:
import zipfile, os

data_dir = '/content/data'
os.makedirs(data_dir, exist_ok=True)

fold_0_path = "/content/drive/MyDrive/fold_0.zip"
val_images_path = "/content/drive/MyDrive/val_images.zip"
csv_path = "/content/drive/MyDrive/C-NMC_test_prelim_phase_data_labels.csv"

with zipfile.ZipFile(fold_0_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(data_dir, 'train'))

with zipfile.ZipFile(val_images_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(data_dir, 'val'))


In [6]:
import pandas as pd
import glob

df = pd.read_csv(csv_path)

def find_image_path(filename, folder):
    matches = glob.glob(f"{folder}/**/{filename}", recursive=True)
    return matches[0] if matches else None

df["file_path"] = df["new_names"].apply(
    lambda name: find_image_path(name, os.path.join(data_dir, 'train')) or
                 find_image_path(name, os.path.join(data_dir, 'val'))
)

df = df.dropna(subset=["file_path"])
df = df.rename(columns={"labels": "label"})


In [7]:
# Extract subject_id from Patient_ID or file_path
df["subject_id"] = df["Patient_ID"].str.extract(r"UID_([A-Za-z0-9]+)_")

from sklearn.model_selection import train_test_split

unique_subjects = df["subject_id"].unique()
train_subjects, test_subjects = train_test_split(unique_subjects, test_size=0.2, random_state=42)

train_df = df[df["subject_id"].isin(train_subjects)].reset_index(drop=True)
test_df = df[df["subject_id"].isin(test_subjects)].reset_index(drop=True)


In [8]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

hf_dataset = {"train": train_dataset, "test": test_dataset}


In [9]:
from transformers import AutoImageProcessor

processor = AutoImageProcessor.from_pretrained("WinKawaks/vit-tiny-patch16-224")


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [10]:
from PIL import Image

def preprocess(example):
    image = Image.open(example["file_path"]).convert("RGB")
    processed = processor(images=image, return_tensors="pt")
    return {
        "pixel_values": processed["pixel_values"].squeeze(),
        "label": int(example["label"])
    }

hf_dataset["train"] = hf_dataset["train"].map(preprocess, remove_columns=hf_dataset["train"].column_names, batched=False)
hf_dataset["test"] = hf_dataset["test"].map(preprocess, remove_columns=hf_dataset["test"].column_names, batched=False)


Map:   0%|          | 0/1334 [00:00<?, ? examples/s]

Map:   0%|          | 0/533 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
    "WinKawaks/vit-tiny-patch16-224",
    num_labels=2,
    ignore_mismatched_sizes=True
)


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([2, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,  # Keep only the best checkpoint
    load_best_model_at_end=True,  # <-- this loads the best model automatically
    metric_for_best_model="accuracy",
    greater_is_better=True,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True
)


In [20]:
from transformers import Trainer, EarlyStoppingCallback
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, preds)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_dataset["train"],
    eval_dataset=hf_dataset["test"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.443200,1.301764,0.733583
2,0.305800,0.865163,0.857411
3,0.287100,0.725183,0.838649
4,0.103800,1.646860,0.776735


TrainOutput(global_step=668, training_loss=0.19212572454304253, metrics={'train_runtime': 733.2812, 'train_samples_per_second': 18.192, 'train_steps_per_second': 2.277, 'total_flos': 2.6625702852919296e+16, 'train_loss': 0.19212572454304253, 'epoch': 4.0})

In [21]:
model.save_pretrained("/content/drive/MyDrive/vit-tiny-leukemia")
processor.save_pretrained("/content/drive/MyDrive/vit-tiny-leukemia")


['/content/drive/MyDrive/vit-tiny-leukemia/preprocessor_config.json']

In [22]:
import torch
from sklearn.metrics import classification_report

model.eval().cuda()  # Make sure model is on GPU

all_preds = []
all_labels = []

for example in hf_dataset["test"]:
    pixel_values = torch.tensor(example["pixel_values"]).unsqueeze(0).half().cuda()  # Move input to GPU + match precision
    with torch.no_grad():
        outputs = model(pixel_values=pixel_values)
        pred = torch.argmax(outputs.logits, dim=1).item()

    all_preds.append(pred)
    all_labels.append(example["label"])

print(classification_report(all_labels, all_preds, target_names=["HEM", "ALL"], digits=2))


              precision    recall  f1-score   support

         HEM       0.61      0.43      0.51        90
         ALL       0.89      0.94      0.92       443

    accuracy                           0.86       533
   macro avg       0.75      0.69      0.71       533
weighted avg       0.84      0.86      0.85       533

